# Models inference

In [1]:
AVAILABLE_MODELS = [
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
]

AVAILABLE_LORA_PATHS = [
    "llama3_8b_instruct/checkpoint-37500",
    "mistral_7b_instruct/checkpoint-8500",
    "qwen_7b_instruct/checkpoint-10000",
]
CSV_FILE = "inference_result/df_test_1.csv"

Check following params for inference

In [2]:
import torch

MAX_NEW_TOKENS = 500
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Llama

In [4]:
use_lora_model = True
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
OUTPUT_DIR = "llama3_8b_instruct/checkpoint-37500"
COLUMN_NAME = "gen_llama_8b_instruct_lora"

In [6]:
# use_lora_model = False
# MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
# COLUMN_NAME = "gen_llama_8b_instruct"

Qwen

In [7]:
# use_lora_model = True
# MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
# OUTPUT_DIR = "qwen_7b_instruct/checkpoint-10000"
# COLUMN_NAME = "gen_qwen_7b_instruct_lora"

In [8]:
# use_lora_model = False
# MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
# OUTPUT_DIR = None
# COLUMN_NAME = "gen_qwen_7b_instruct"

Mistral

In [9]:
# use_lora_model = True
# MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3"
# OUTPUT_DIR = "mistral_7b_instruct/checkpoint-8500"
# COLUMN_NAME = "gen_mistral_7b_instruct_lora"

In [10]:
# use_lora_model = False
# MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3"
# COLUMN_NAME = "gen_mistral_7b_instruct"

In [5]:
llama_instuct_prompt = """Ты профессиональный грамотный редактор новостных статей. Пожалуйста, сохраняй факты из заголовка статьи. Напиши только статью и ничего больше.
Напиши небольшую захватывающую новостную статью, заголовком которой мог бы быть:

{}

Текст статьи:
"""

llama_instuct_lora_prompt = """Напиши новостную статью, заголовком которой мог бы быть:
{}

Текст статьи:
"""

mistral_instruct_prompt = """Ты русскоязычный редактор новостных статей. Используй только русский язык! Пожалуйста, сохраняй факты из заголовка статьи.
Напиши захватывающую новостную статью, заголовком которой мог бы быть:

{}

Текст статьи:
"""

qwen_instruct_prompt = """Ты русскоязычный редактор новостных статей. Используй только русский язык! Пожалуйста, сохраняй факты из заголовка статьи.
Напиши захватывающую новостную статью, заголовком которой мог бы быть:

{}

Текст статьи:
"""

prompt_dict = {
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit": llama_instuct_lora_prompt if use_lora_model else llama_instuct_prompt,
    "unsloth/mistral-7b-instruct-v0.3": mistral_instruct_prompt,
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit": qwen_instruct_prompt,
}

In [11]:
PROMPT = prompt_dict[MODEL_NAME]

In [12]:
assert MODEL_NAME in AVAILABLE_MODELS
assert OUTPUT_DIR in AVAILABLE_LORA_PATHS

# Functions to load and delete model

load `model`, `tokenizer`

In [13]:
import unsloth
import torch

from unsloth import FastLanguageModel

def load_model(use_lora_model=False):
    if use_lora_model:
        model_name = OUTPUT_DIR
        assert model_name in AVAILABLE_LORA_PATHS
    else:
        model_name = MODEL_NAME
        assert model_name in AVAILABLE_MODELS

    max_seq_length = 2048
    load_in_4bit = True
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, 
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)
    return model, tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [14]:
import gc

def free_gpu():
    global model
    
    print("Memory before del model: ", torch.cuda.memory_allocated() // (1024 ** 3) )
    
    del model
    
    gc.collect()
    torch.cuda.empty_cache()

    print("Memory after del model: ", torch.cuda.memory_allocated() // (1024 ** 3) )

# Functions for batch inference

Works for models from `AVAILABLE_MODELS`

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from tqdm import tqdm as tqdm_a

BATCH_SIZE = 8
MAX_LENGTH = 2048
MAX_NEW_TOKENS = 2048


class PromptDataset(Dataset):
    def __init__(self, tokenizer, prompt, csv_path):
        self.data = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.prompt = prompt
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.prompt.format(self.data.iloc[idx]['cleaned_title'])

def collate_fn(batch, tokenizer):
    return tokenizer(
        batch,
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt",
        padding_side="left",
    )


def batch_inference(model, tokenizer, dataloader, generation_config):
    all_responses = []
    with torch.no_grad():
        for batch in tqdm_a(dataloader):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                generation_config=generation_config
            )
            
            responses = tokenizer.batch_decode(
                outputs[:, input_ids.shape[1]:], 
                skip_special_tokens=True
            )
            all_responses.extend(responses)
    return all_responses

# Multilingual models and LoRA

## Batch inference

Uses `MODEL_NAME`, `OUTPUT_DIR` and `use_lora_model` params to load specific model

In [16]:
model, tokenizer = load_model(use_lora_model)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.621 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [17]:
generation_config = GenerationConfig(
    temperature=0.6,            # Контроль случайности (0.0-1.0, больше - креативнее)
    repetition_penalty=1.2,     # Штраф за повторения (1.0 - нет штрафа, >1.0 - меньше повторов)
    top_p=0.95,                 # Фильтрация ядра (nucleus sampling)
    do_sample=True,             # Включить стохастическое семплирование
    max_new_tokens=MAX_NEW_TOKENS,        # Максимальное количество новых токенов
    pad_token_id=tokenizer.eos_token_id 
)

dataset = PromptDataset(tokenizer, PROMPT, CSV_FILE)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)

all_responses = batch_inference(model, tokenizer, dataloader, generation_config)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [05:00<00:00, 12.03s/it]


In [114]:
print(all_responses[0])

В Петербурге тушили пожар в гаражах. На месте работали 4 единицы техники и 15 человек личного состава.
Как сообщает ГУ МЧС по городу, информация о возгорании поступила на пульт дежурного службы "01" около полудня 14 августа от дома No10/1 по Звездной улице. Огнем были охвачены шесть металлических гаражей, в которых находились два автомобиля "Мерседес", три ВАЗ-2105, двухместный мотоцикл "Ямаха", а также различные строительные материалы.
Пожару присвоили повышенный номер сложности 1-бис, из-за чего было задействовано дополнительное подразделение, которое направилось к месту происшествия с другого участка города.


## Save generated texts

In [116]:
df_test = pd.read_csv(CSV_FILE)
df_test.head()

,url,text,title,source,timestamp,cleaned_title,cleaned_text,len_cleaned_text
0,http://www.fontanka.ru/2013/10/16/017/,Пожар произошел минувшей ночью в Московском ра...,На стоянке на Звездной обгорело семь машин,taiga_fontanka,1381911180,На стоянке на Звездной обгорело семь машин,Пожар произошел минувшей ночью в Московском ра...,476
1,https://tass.ru/sport/3866330,"МОСКВА, 13 декабря. /ТАСС/. Олимпийская чемпио...",Олимпийская чемпионка Елена Веснина стала поче...,ods_tass,1481617927,Олимпийская чемпионка Елена Веснина стала поче...,Олимпийская чемпионка по теннису россиянка Еле...,851
2,http://rusplt.ru/region-news/barnaul/v-altaysk...,С начала ноября четыре жителя Алтайского края ...,В Алтайском крае под лед провалились четыре че...,buriy,1447147996,В Алтайском крае под лед провалились четыре че...,С начала ноября четыре жителя Алтайского края ...,744
3,https://lenta.ru/news/2002/08/06/chisinau/,Президент Молдавии Владимир Воронин своим указ...,Президент Молдавии уволил главу своей службы о...,lenta,1028592000,Президент Молдавии уволил главу своей службы о...,Президент Молдавии Владимир Воронин своим указ...,697
4,https://lenta.ru/news/2017/08/21/annabel/,Посетительнице кинотеатра в Бразилии стало пло...,Бразильянке вызвали скорую после просмотра хор...,lenta,1503273600,Бразильянке вызвали скорую после просмотра хор...,Посетительнице кинотеатра в Бразилии стало пло...,655


In [117]:
df_test[COLUMN_NAME] = all_responses
df_test.to_csv(CSV_FILE, index=False)

free_gpu()

# Ru language models

## YandexGPT inference

https://huggingface.co/yandex/YandexGPT-5-Lite-8B-instruct

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-instruct"
COLUMN_NAME = "gen_yagpt_8b"
PROMPT = """Напиши захватывающую новостную статью, заголовком которой мог бы быть:

{}

Текст статьи:
"""

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    torch_dtype="auto",
)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
BATCH_SIZE = 4

dataset = PromptDataset(tokenizer, PROMPT, CSV_FILE)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)

generation_config = GenerationConfig(
    temperature=0.6,           # Контроль случайности (0.0-1.0, больше - креативнее)
    repetition_penalty=1.2,    # Штраф за повторения (1.0 - нет штрафа, >1.0 - меньше повторов)
    top_p=0.9,                 # Фильтрация ядра (nucleus sampling)
    do_sample=True,            # Включить стохастическое семплирование
    max_new_tokens=MAX_NEW_TOKENS,        # Максимальное количество новых токенов
)

In [18]:
dataset = PromptDataset(tokenizer, PROMPT, CSV_FILE)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)

all_responses = batch_inference(model, tokenizer, dataloader, generation_config)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:21<00:00,  6.44s/it]


In [21]:
print(all_responses[1])


Елена Веснина – знаменитая российская теннисистка и олимпийская чемпионка. Она была удостоена высокого звания «Почетный гражданин города-курорта Сочи». Это признание заслуг спортсменки перед городом и всей страной в целом. 

Веснину знают во всем мире благодаря ее выдающимся достижениям на Олимпийских играх. В составе российской сборной она завоевала золото в парном разряде по теннису. Этот успех стал настоящим праздником для всех поклонников спорта и гордостью для страны.

На торжественной церемонии вручения награды Елене Весниной были вручены символы этого почётного статуса - лента с гербом города и удостоверение Почётного гражданина. Глава города отметил, что выбор пал именно на Елену неслучайно: её спортивные успехи стали символом силы воли и настойчивости, которые помогают достигать самых высоких целей.

Сама же Елена выразила глубокую благодарность за оказанное ей доверие и подчеркнула важность поддержки со стороны общества для спортсменов. "Это большая честь для меня — стать По

In [23]:
df_test = pd.read_csv(CSV_FILE)
df_test[COLUMN_NAME] = all_responses
df_test.to_csv(CSV_FILE, index=False)

In [24]:
free_gpu()

Memory before del model:  14
Memory after del model:  0


## Sber model inference

https://huggingface.co/ai-forever/ruGPT-3.5-13B


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "ai-forever/ruGPT-3.5-13B"
COLUMN_NAME = "gen_rugpt_13b"
PROMPT = """Заголовок захватывающей подробной длинной новостной статьи:
{}

Текст статьи:
"""
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
generation_config = GenerationConfig(
    temperature=0.6, 
    repetition_penalty=1.2, 
    top_p=0.9, 
    do_sample=True, 
    max_new_tokens=500, 
)

In [16]:
df_test = pd.read_csv(CSV_FILE)

def get_generated_text(generated_text):
    # for sber model only
    if "\n" in generated_text:
        text = generated_text[:generated_text.find("\n")]
    return text

    
generated_texts = []
for title in tqdm_a(df_test["cleaned_title"].values):
    prompt = PROMPT.format(title)
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(DEVICE)

    outputs = model.generate(
        **inputs,
        generation_config=generation_config
    )

    generated_text = tokenizer.decode(
        outputs[0, inputs["input_ids"].shape[1]:], 
        skip_special_tokens=True
    )
    generated_text_no_prompt = get_generated_text(generated_text)

    print(f"Title:\n{title}")
    print(f"Text:\n{generated_text_no_prompt}")
    print()
    print("-" * 50)
    print()

    generated_texts.append(generated_text_no_prompt)

  0%|▌                                                                                                       | 1/200 [00:16<55:26, 16.72s/it]

Title:
На стоянке на Звездной обгорело семь машин
Text:
Сегодня ночью, около 2 часов, в районе дома №5 по улице Звездная произошло возгорание семи автомобилей. На месте происшествия работают сотрудники МЧС и полиции...

--------------------------------------------------



  1%|█                                                                                                       | 2/200 [00:33<54:59, 16.67s/it]

Title:
Олимпийская чемпионка Елена Веснина стала почетным гражданином Сочи
Text:
В Сочи состоялось торжественное вручение паспортов граждан России четырнадцати юным сочинцам, в том числе и Елене Весниной. Олимпийская чемпионка по теннису получила паспорт вместе с золотой медалью за победу на Олимпиаде-2014, а также дипломом Почетного гражданина города-курорта Сочи. Вместе со спортсменкой паспорта получили ее родители – заслуженный тренер СССР Александр Волков и олимпийский призер Светлана Кузнецова. Также удостоверения были выданы другим известным жителям Сочи - президенту Федерации спортивной борьбы Михаилу Мамиашвили и актеру Владимиру Машкову. 

--------------------------------------------------



  2%|█▌                                                                                                      | 3/200 [00:50<54:46, 16.68s/it]

...

Title:
11 октября открывается выставка Health&amp;Medical Tourism
Text:
Выставка Health &amp; Medical Tourism будет проходить в выставочном комплексе «Экспоцентр» с 11 по 13 октября. Это мероприятие проводится под эгидой Всемирной организации здравоохранения (ВОЗ). В рамках выставки состоится конференция, посвященная проблемам здоровья и медицинской индустрии во всем мире. На конференции будут обсуждаться такие вопросы как роль туризма для оказания медицинских услуг, использование передовых технологий в области медицины, а также перспективы развития медицинского туризма на международном уровне. 

--------------------------------------------------



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████▍| 199/200 [55:28<00:16, 16.73s/it]

Title:
Семьи восьми погибших при взрывах на складе в Забайкалье получили по 2 млн рублей
Text:
Власти выплатили семьям пострадавших и погибших при взрыве боеприпасов в Забайкальском крае компенсации, сообщает пресс-служба губернатора региона. Семьям погибших из бюджета края выделили по два миллиона рублей (около 100 тысяч долларов), пострадавшим - от 200 до 400 тысяч рублей (от 10 до 30 тысяч долларов). Кроме того, власти организовали для семей бесплатные похороны. В общей сложности было выделено семь миллионов рублей. Еще шесть миллионов будут направлены на оказание помощи пострадавшим. Взрыв произошел утром 12 августа неподалеку от села Большая Тура Карымского района. Погибшими числятся восемь человек, среди них трое детей. Ранения различной степени тяжести получили более 40 местных жителей.

--------------------------------------------------



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [55:44<00:00, 16.72s/it]

Title:
Все подразделения МЧС Камчатки приведены в состояние готовности в связи с циклоном
Text:
В Петропавловске-Камчатском, Елизово и Вилючинске сегодня объявлено штормовое предупреждение. Об этом сообщили ИТАР ТАСС в Камчатской гидрометеослужбе. По данным синоптиков, на полуострове ожидаются сильные осадки (до 10 мм), ветер порывами до 30 м/с. Кроме того, ожидается метель со снегом.

--------------------------------------------------



In [14]:
request = "Стих про программиста может быть таким:"

encoded_input = tokenizer(request, return_tensors='pt', \
                          add_special_tokens=False).to('cuda:0')
output = model.generate(
    **encoded_input,
    num_beams=2,
    do_sample=True,
    max_new_tokens=100
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Стих про программиста может быть таким:

Программист с утра сидит в туалете,
Программист с утра сидит в туалете,
Он весь в говне, он весь в говне,
В говне он весь, в говне.

А в это время, в это время,
А в это время, в это время,
Программист с утра сидит в туалете,
Программист с утра сидит в туалете,
Он весь в говне, он весь в говне,



In [18]:
df_test = pd.read_csv(CSV_FILE)
df_test[COLUMN_NAME] = generated_texts
df_test.to_csv(CSV_FILE, index=False)

In [28]:
generated_texts[9]

'Малайзийские спасатели сегодня утром извлекли из моря два больших фрагмента самолета авиакомпании AirAsia, который потерпел крушение 28 декабря. Обломки были обнаружены к западу от острова Борнео примерно в 200 км от побережья штата Сабах на глубине 32-35 м. "На данный момент мы не можем сказать точно, принадлежат ли найденные фрагменты пропавшему самолету", - заявил глава Национального поисково-спасательного управления (Basarnas) Индонезии Хенри Бамбанг Сулистьо. По его словам, обломки очень большие и могут принадлежать только лайнеру А320. На одном из них виден большой логотип Malaysian Airlines, а другой имеет бортовой номер 9M-MAK. В районе обнаружения обломков работают четыре судна индонезийских ВМС. Спасателям удалось обнаружить несколько тел погибших при катастрофе. Тем временем поиски пропавшего авиалайнера продолжаются уже почти неделю. За это время они обследовали около 3 тыс кв.км акватории Индийского океана и более 2 тыс кв.км воздушного пространства над ним, однако следов

In [ ]:
free_gpu()

## T-bank model inference

https://huggingface.co/t-tech/T-lite-it-1.0

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "t-tech/T-lite-it-1.0"
COLUMN_NAME = "gen_tliteit_7b"
PROMPT = """Заголовок захватывающей подробной длинной новостной статьи:
{}

Текст статьи:
"""
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
def get_generated_text(text):
    # for t-lite model only
    prefix = "Текст статьи:"
    if prefix in text:
        text = text[text.find(prefix):]
        text = text[text.find("\n"):]
    text = text[:text.rfind('.') + 1]
    return text.strip()

generated_texts = []
for title in tqdm_a(df_test["cleaned_title"].values):
    prompt = PROMPT.format(title)
    messages = [
        {"role": "system", "content": "Ты профессиональный редактор статей. Твоя задача - быть полезным ассистентом."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=500,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_text = get_generated_text(generated_text)
    
    # ---
    print(f"Title:\n{title}")
    print(f"Text:\n{generated_text}")
    print()
    print("-" * 50)
    print()

    generated_texts.append(generated_text)


  0%|▌                                                                                                     | 1/200 [00:19<1:05:07, 19.63s/it]

Title:
На стоянке на Звездной обгорело семь машин
Text:
Вчера вечером на одной из крупнейших парковок в районе Звездной произошел серьезный инцидент — сгорели сразу семь автомобилей. Информация о пожаре поступила в службу спасения около 21:00, и к месту происшествия были немедленно направлены пожарные расчеты.

По предварительным данным, причиной пожара стала короткое замыкание в электропроводке одного из автомобилей. Однако официальное расследование продолжается, и власти пока не исключают и другие возможные причины, такие как поджог или техническая неисправность системы безопасности стоянки.

Пожар начался в одном из отдаленных уголков парковки, где находилось несколько автомобилей, припаркованных в ряд. Огонь быстро распространился из-за сильного ветра и плотной застройки территории. Пострадавшие автомобили оказались полностью уничтожены огнем, а некоторые из них получили повреждения и от близости к очагу возгорания.

Свидетели происшествия сообщают, что видели дым задолго до того, 

  1%|█                                                                                                     | 2/200 [00:39<1:05:19, 19.79s/it]

Title:
Олимпийская чемпионка Елена Веснина стала почетным гражданином Сочи
Text:
Вчера, 15 октября, олимпийская чемпионка по теннису Елена Веснина была удостоена звания почетного гражданина города Сочи. Это событие стало важным шагом в признании её выдающихся достижений и значительного вклада в развитие спорта в регионе.

Елена Веснина, родившаяся и выросшая в Сочи, сделала себе имя на международной арене, завоевав золотую медаль на Олимпийских играх 2014 года в составе женской сборной России. Её победа стала символом силы духа и упорства, вдохновляя молодое поколение спортсменов не только в Сочи, но и по всей стране.

После завершения карьеры Елена активно занимается развитием тенниса в своем родном городе. Она основала детскую теннисную академию, которая уже успела подготовить множество талантливых юных игроков. Кроме того, Веснина активно участвует в благотворительных мероприятиях, поддерживая детей с ограниченными возможностями и помогая им развивать свои спортивные навыки.

Получе

  2%|█▌                                                                                                    | 3/200 [00:59<1:04:42, 19.71s/it]

...

Title:
11 октября открывается выставка Health&amp;Medical Tourism
Text:
11 октября ведущие медицинские центры, туристические компании и эксперты со всего мира соберутся на престижную выставку Health & Medical Tourism, чтобы представить последние достижения в области медицинского туризма и обсудить перспективы его развития.

### Введение
Медицинский туризм — это стремительно развивающаяся отрасль, которая объединяет медицину и путешествия, предоставляя пациентам доступ к высококачественным медицинским услугам в комфортных условиях. Выставка Health & Medical Tourism 2023 года станет важной платформой для обмена опытом и знаниями, способствуя укреплению международного сотрудничества в этой сфере.

### Основные темы выставки
- **Инновации в медицине:** На выставке будут представлены передовые технологии и методики лечения, которые уже сегодня меняют подходы к здравоохранению.
- **Качество и безопасность:** Обсуждение стандартов качества и безопасности медицинских услуг, а также их влияние 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████▌| 199/200 [1:05:01<00:19, 19.65s/it]

Title:
Семьи восьми погибших при взрывах на складе в Забайкалье получили по 2 млн рублей
Text:
В результате трагических событий, произошедших на складе в Забайкальском крае, семьи восьми погибших получили значительные компенсации. Каждая семья получила по 2 миллиона рублей, что стало первым шагом в оказании поддержки близким погибших.

**Что произошло?**  
19 июля 2023 года на одном из складов в городе Чита произошел мощный взрыв, в результате которого погибли восемь человек, а еще более десятка получили ранения различной степени тяжести. По предварительным данным, причиной взрыва стала утечка газа, однако расследование продолжается для установления точных обстоятельств происшествия.

**Какие меры предприняты?**  
Правительство региона немедленно приступило к оказанию помощи пострадавшим и их семьям. Помимо материальной компенсации, были организованы психологические консультации для тех, кто пережил травматический опыт. Расследование инцидента поручено специальной комиссии, которая раб

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [1:05:21<00:00, 19.61s/it]

Title:
Все подразделения МЧС Камчатки приведены в состояние готовности в связи с циклоном
Text:
**Все подразделения МЧС Камчатки приведены в состояние готовности в связи с циклоном: как регион готовится к возможным последствиям**

На фоне приближающегося циклона, который может вызвать значительные погодные изменения и потенциальные угрозы для населения и инфраструктуры, все подразделения МЧС на Камчатке были приведены в состояние повышенной готовности. Специалисты проводят тщательную подготовку, чтобы минимизировать риски и обеспечить безопасность жителей региона.

### Подготовка к экстренным ситуациям

В рамках подготовки к возможным неблагоприятным погодным условиям, сотрудники МЧС провели проверку всех аварийно-спасательных служб и оборудования. Были проведены тренировки по эвакуации населения из зон возможного затопления и ликвидации последствий штормовых волн. Важным аспектом является также работа с местными органами власти и общественными организациями для координации действий в 

In [42]:
print(generated_texts[0])

Вчера вечером на одной из крупнейших парковок в районе Звездной произошел серьезный инцидент — сгорели сразу семь автомобилей. Информация о пожаре поступила в службу спасения около 21:00, и к месту происшествия были немедленно направлены пожарные расчеты.

По предварительным данным, причиной пожара стала короткое замыкание в электропроводке одного из автомобилей. Однако официальное расследование продолжается, и власти пока не исключают и другие возможные причины, такие как поджог или техническая неисправность системы безопасности стоянки.

Пожар начался в одном из отдаленных уголков парковки, где находилось несколько автомобилей, припаркованных в ряд. Огонь быстро распространился из-за сильного ветра и плотной застройки территории. Пострадавшие автомобили оказались полностью уничтожены огнем, а некоторые из них получили повреждения и от близости к очагу возгорания.

Свидетели происшествия сообщают, что видели дым задолго до того, как огонь перекинулся на соседние машины. Некоторые из в

In [43]:
df_test = pd.read_csv(CSV_FILE)
df_test[COLUMN_NAME] = generated_texts
df_test.to_csv(CSV_FILE, index=False)

In [ ]:
free_gpu()